In [1]:
from itertools import product

import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import gurobipy_pandas as gppd

In [2]:
I = ['a', 'b', 'c'] # List of facilities
W = [1,2,3,4,5] # List of weeks

P_initialWt = pd.Series({'a': 0.1, 'b': 0.3, 'c': 0.2})
P_initialWt.index.name = 'Facility'
data = pd.DataFrame(list(product(I, W)), columns=['Facility', 'Week']).set_index(["Facility","Week"])

m = gp.Model("Model1")
CV_EndInv = gppd.add_vars(m, data, name="end_inv", lb=0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS)
CV_StartInv = gppd.add_vars(m, data, name="start_inv", lb=0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS)

m.update()

Restricted license - for non-production use only - expires 2024-10-28


Constraint I am trying to model in pandas vectorized form is `CV_StartInv[i,w] = CV_EndInv[i,w-1]` if `w > 1` else `P_InitialWt[i]`, for all i in I and w in W.

In [3]:
# Shift 
shifted = CV_StartInv.reset_index().assign(Week=lambda df: df['Week']+1).set_index(["Facility","Week"])
lhs, rhs = shifted.align(CV_EndInv, axis=0, join='inner')
lhs

start_inv
Facility Week                             
a        2     <gurobi.Var start_inv[a,1]>
         3     <gurobi.Var start_inv[a,2]>
         4     <gurobi.Var start_inv[a,3]>
         5     <gurobi.Var start_inv[a,4]>
b        2     <gurobi.Var start_inv[b,1]>
         3     <gurobi.Var start_inv[b,2]>
         4     <gurobi.Var start_inv[b,3]>
         5     <gurobi.Var start_inv[b,4]>
c        2     <gurobi.Var start_inv[c,1]>
         3     <gurobi.Var start_inv[c,2]>
         4     <gurobi.Var start_inv[c,3]>
         5     <gurobi.Var start_inv[c,4]>

In [4]:
# Shift the end variables so that 'w' in the index aligns with the 'w-1' variable
lhs = CV_EndInv.groupby("Facility").transform(lambda s: s.shift(1)).dropna()
# Extract only the 'w > 1' part of the start variables
rhs = CV_StartInv.loc[:, 2:]
# Both series are now on the index w = 2..5, so can be aligned
balance = gppd.add_constrs(m, lhs, GRB.EQUAL, rhs, name="Balance")

m.update()
balance.apply(m.getRow)

Facility  Week
a         2       end_inv[a,1] + -1.0 start_inv[a,2]
          3       end_inv[a,2] + -1.0 start_inv[a,3]
          4       end_inv[a,3] + -1.0 start_inv[a,4]
          5       end_inv[a,4] + -1.0 start_inv[a,5]
b         2       end_inv[b,1] + -1.0 start_inv[b,2]
          3       end_inv[b,2] + -1.0 start_inv[b,3]
          4       end_inv[b,3] + -1.0 start_inv[b,4]
          5       end_inv[b,4] + -1.0 start_inv[b,5]
c         2       end_inv[c,1] + -1.0 start_inv[c,2]
          3       end_inv[c,2] + -1.0 start_inv[c,3]
          4       end_inv[c,3] + -1.0 start_inv[c,4]
          5       end_inv[c,4] + -1.0 start_inv[c,5]
Name: Balance, dtype: object

In [5]:
# For the initial constraints, we only need to align the facilities
initial = gppd.add_constrs(
    m,
    CV_StartInv.loc[:, 1],  # All facilities, time step 1
    GRB.EQUAL,
    P_initialWt,
    name="Initial",
)

m.update()
initial.apply(m.getRow)

Facility
a    start_inv[a,1]
b    start_inv[b,1]
c    start_inv[c,1]
Name: Initial, dtype: object